<a href="https://colab.research.google.com/github/borhanDZ/your-code-is-vulnerable/blob/master/asma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp38-cp38-linux_x86_64.whl size=4438485 sha256=e80638ef3fea3c985be5ce649442d8b263fbcd675cbdd26f85460617f1caaa02
  Stored in directory: /root/.cache/pip/wheels/f3/35/48/0b9a7076995eea5ea64a7e4bc3f0f342f453080795276264e7
Successfully built mpi4py


In [ ]:
%%writefile asma.py
from mpi4py import MPI
from random import randint

print ("######## Tri a bulle with Point-to-Point Communication #########")

def tri(L,sp,fp,V):
    for i in range(1,V):
        for x in range(sp,fp-i):
            if L[x]> L[x+1]:
                e,L[x+1] = L[x+1],L[x]
                L[x] = e 
    return L

def tri_every_party(paralel_v):
    L,comm,P,size = paralel_v
    data = [0,0,0,0]
    V = len(L[0])//size  # N/P 
    for i in range(size):
        if P == 0:
           for j in range(len(L)):
              data[j] = tri(L[j],0,V,V)   # data[j]: indicate the L list share between process
              comm.send(data[j], dest=1,tag=j+1)
        if P == 1:
           for j in range(4):
              data[j] = comm.recv(source=0,tag=j+1)
              data[j] = tri(data[j],V,(2*V),V)  
              comm.send(data[j], dest=2,tag=j+1)
        if P == 2:
           for j in range(4):
              data[j] = comm.recv(source=1,tag=j+1)
              data[j] = tri (data[j],(2*V),(3*V),V)
              comm.send(data[j], dest=3,tag=j+1)
        if P == 3:
           for j in range(4):
              data[j] = comm.recv(source=2,tag=j+1)
              data[j] = tri(data[j],(3*V),(4*V),V)
              print("Every process try to sort his party:",data[j],"\n")
              if j == 3:
                 print("\n\n")
                 return data
           
def change_min_max(L,Tup,Pv,comm):
    x = Tup
    #comm.send(L,dest=Pv,tag=Pv)
    #L = comm.recv(source=Pv,tag=Pv)
    for i in range(1,x):
       for k in range(len(L)-i):
        if L[k]>L[k+1]:
            E = L[k]
            L[k] = L[k+1]
            L[k+1] = E
    return L

def paralell_tri(paralel_v):
    L,comm,P,size = paralel_v
    V = len(L[0])//size  # N/P
    data = tri_every_party(paralel_v) 
    if data != None:
       for j in range(len(data)):

          while True:

              print ("jsdfhkgv",P)
              if P == 0:
                Pvoisin = 1
                Tup = (0,V,V)
                data[j] = change_min_max(data[j],Tup,Pvoisin,comm)
              if P ==1:
                Pvoisin = 2
                Tup = (V,2*V,V)
                data[j] = change_min_max(data[j],Tup,Pvoisin,comm)
              if P ==2:
                Pvoisin = 3
                Tup = (2*V,3*V,V)
                data[j] = change_min_max(data[j],Tup,Pvoisin,comm)
              if P == 3:
                Pvoisin = 0
                Tup = (4*V)
                result = change_min_max(data[j],Tup,Pvoisin,comm)
                
              if (max(result[0:V])<= min(result[V:2*V]) and max(result[V:2*V])<= min(result[2*V:3*V]) and max(result[2*V:3*V])<= min(result[3*V:4*V])):
                print("Congratulation All 4 process has been doing the work correctly:\n",result,"\n")
                break
       

#... this func generate matrix l[n][m]
def gen_rand_list(n,m):
    Gl,Pl = [],[]
    for a in range(n):
        for b in range(m):
            Pl.append(randint(0,30))
        Gl.append(Pl) 
        Pl = []
    return Gl


def main():
    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    P = comm.Get_rank()
    r = gen_rand_list(4,16)    #[12,5,11,2,8,7,19,4,3,10,15,9,20,14,6,1]
    paralel_v = (r,comm,P,size)
    print(paralell_tri(paralel_v))

main()

Writing asma.py


In [ ]:
! mpirun --allow-run-as-root -np 4 python asma.py

######## Tri a bulle with Point-to-Point Communication #########
None
######## Tri a bulle with Point-to-Point Communication #########
######## Tri a bulle with Point-to-Point Communication #########
######## Tri a bulle with Point-to-Point Communication #########
None
None
Every process try to sort his party: [2, 4, 9, 27, 5, 16, 19, 29, 5, 14, 17, 23, 3, 6, 14, 22] 

Every process try to sort his party: [5, 24, 24, 25, 6, 21, 23, 23, 25, 27, 28, 30, 4, 5, 21, 25] 

Every process try to sort his party: [2, 8, 29, 29, 2, 11, 11, 19, 1, 8, 10, 16, 3, 19, 21, 25] 

Every process try to sort his party: [1, 8, 22, 28, 4, 23, 28, 30, 7, 11, 18, 29, 6, 20, 22, 25] 




jsdfhkgv 3
Congratulation All 4 process has been doing the work correctly:
 [2, 3, 4, 5, 5, 6, 9, 14, 14, 16, 17, 19, 22, 23, 27, 29] 

jsdfhkgv 3
Congratulation All 4 process has been doing the work correctly:
 [4, 5, 5, 6, 21, 21, 23, 23, 24, 24, 25, 25, 25, 27, 28, 30] 

jsdfhkgv 3
Congratulation All 4 process has been doin